In [1]:
# import zipfile
# my_zip = zipfile.ZipFile('drive-download-20220716T023838Z-001.zip')
# my_zip.extractall()

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from ccdproc import CCDData, trim_image, subtract_bias, subtract_dark, flat_correct, Combiner

import astropy.units as u
from astropy.visualization import simple_norm, ImageNormalize, ZScaleInterval

import warnings
warnings.simplefilter("ignore")

import glob
import os
from scipy.signal import medfilt

from photutils import aperture_photometry, DAOStarFinder, CircularAperture, CircularAnnulus
from astropy.stats import sigma_clipped_stats

from astropy.modeling.models import Moffat1D

## Read in data

In [17]:
# data #
# pg1633 (standard star)
pg1633_loc = glob.glob('C:/Users/josue/Work-space/data/07082022/P[gG]1633*.fits')
pg1633_loc = [loc for loc in pg1633_loc if 'test' not in loc]

print(pg1633_loc)
# pg1633 = []
# for x in pg1633_loc:
#     image = CCDData.read(x, unit='adu')
#     pg1633.append(image)

# # pg1648 (standard star)
# pg1648_loc = glob.glob('./07*/P[gG]1648*.fits')
# pg1648 = []
# for x in pg1648_loc:
#     image = CCDData.read(x, unit='adu')
#     pg1648.append(image)
    
# # g9348 (standard star)
# g9348_loc = glob.glob('./07*/[gG]9348*.fits')
# g9348 = []
# for x in g9348_loc:
#     image = CCDData.read(x, unit='adu')
#     g9348.append(image)

# # lilsaldalbary (extinction star; BD+234617)
# lilsal_loc = glob.glob('./07*/lilsa[ld]*.fits')
# lilsal = []
# for x in lilsal_loc:
#     image = CCDData.read(x, unit='adu')
#     lilsal.append(image)

# # ngc7318 (don't need... delete?)
# ngc7318_loc = glob.glob('./07*/NGC7318*.fits')
# ngc7318 = []
# for x in ngc7318_loc:
#     image = CCDData.read(x, unit='adu')
#     ngc7318.append(image)

[]


In [18]:
# biases #
bias = []
bias_07 = glob.glob('./07072022/bias*.fits')
bias_08 = glob.glob('./07082022/bias*.fits')

for x in bias_07:
    image = CCDData.read(x, unit='adu')
    bias.append(image)

for x in bias_08:
    image = CCDData.read(x, unit='adu')
    bias.append(image)

In [5]:
# sky flats
skyflats = []
skyflats_loc = glob.glob('./07*/sky*.fits')
skyflats_loc = [loc for loc in skyflats_loc if 'test' not in loc]

for x in skyflats_loc:
    image = CCDData.read(x, unit='adu')
    skyflats.append(image)

In [6]:
# dome flats
domeflats = []
domeflats_loc = glob.glob('./07*/*dome*.fits')
domeflats_loc = [loc for loc in domeflats_loc if 'test' not in loc]

for x in domeflats_loc:
    image = CCDData.read(x, unit='adu')
    domeflats.append(image)

In [7]:
# linearity tests
lintests = []
lintests_08 = glob.glob('./07082022/lintest*.fits')

for x in lintests_08:
    image = CCDData.read(x, unit='adu')
    lintests.append(image)

## _CCD Characterization_

In [8]:
# dunno how to sort array by header; tidy up later
exfil_flats = []
for x in skyflats:
    y = [x.header['exptime'], x.header['filter']]
    exfil_flats.append(y)
    
exfil_flats

[]

### Gain Calculation

In [9]:
flat1 = skyflats[3]
flat2 = skyflats[8]
bias1 = bias[0]
bias2 = bias[1]

IndexError: list index out of range

In [ ]:
np.mean(flat1)

In [ ]:
mean_f1 = np.mean(flat1)
print(mean_f1)

mean_f2 = np.mean(flat2)
print(mean_f2)

mean_b1 = np.mean(bias1)
print(mean_b1)

mean_b2 = np.mean(bias2)
print(mean_b2)

sigma_f1f2 = np.std(flat1.subtract(flat2))
print(sigma_f1f2)

sigma_b1b2 = np.std(bias1.subtract(bias2))
print(sigma_f1f2)

gain = ((mean_f1+mean_f2)-(mean_b1+mean_b2))/(sigma_f1f2**2+sigma_b1b2**2)
print(gain)
print(f'\nGain is {gain:.3f} e-/ADU')

### Read Noise Calculation

~is this even correct~

In [ ]:
read_noise = gain*sigma_b1b2/np.sqrt(2)

print(read_noise)
print(f'\nRead noise is {read_noise:.3f} e-')

## _Linearity Test_

In [ ]:
exp_times = []
means = []

for x in lintests:
    exptime = x.header['exptime']
    exp_times.append(exptime)
    print('Exposure time {} sec'.format(exptime))

    mean_counts = np.mean(x)
    print('Mean counts: {:.2f}\n'.format(mean_counts))
    means.append(mean_counts)

# sorting by exp time
exp_times = np.array(exp_times)
means = np.array(means)

time_sort = np.argsort(exp_times)
exp_times = exp_times[time_sort]
means = means[time_sort]

In [ ]:
plt.figure(figsize=(10, 8))
plt.rcParams['font.size'] = 16

plt.plot(exp_times, means, '-ko')
plt.plot(4.5, 59800, 'rx', mew=1.5, markersize=12, label='Max linearity point?')
plt.plot(1.2, 16300, 'bx', mew=1.5, markersize=12, label='Min linearity point?')
plt.title('Linearity Test')
plt.xlabel('Exposure Time (sec)')
plt.ylabel('Mean counts')
plt.legend()
plt.savefig('lin_plot')

## _Removing Instrumental Signature+Illumination Correction_

### Removing Bias

In [ ]:
biases = Combiner(bias)
master_bias = biases.median_combine()
master_bias.header = bias[0].header
master_bias.header['Combined'] = 'Combined with biases from 07 and 08'

plt.figure(figsize=(5,5))
plt.imshow(master_bias, vmin=np.percentile(master_bias, 5),vmax=np.percentile(master_bias, 95), 
          cmap='gray', interpolation='nearest');

In [ ]:
np.mean(master_bias)

In [ ]:
# data bias subtraction
pg1633_bias = []
for file in pg1633:
    bias = subtract_bias(file, master_bias, add_keyword=True)
    pg1633_bias.append(bias)

pg1648_bias = []
for file in pg1648:
    bias = subtract_bias(file, master_bias, add_keyword=True)
    pg1648_bias.append(bias)
    
g9348_bias = []
for file in g9348:
    bias = subtract_bias(file, master_bias, add_keyword=True)
    g9348_bias.append(bias)

lilsal_bias = []
for file in lilsal:
    bias = subtract_bias(file, master_bias, add_keyword=True)
    lilsal_bias.append(bias)

ngc7318_bias = []
for file in ngc7318:
    bias = subtract_bias(file, master_bias, add_keyword=True)
    ngc7318_bias.append(bias)

In [ ]:
# sky flats bias subtraction
skyflats_bias = []
for file in skyflats:
    bias = subtract_bias(file, master_bias, add_keyword=True)
    skyflats_bias.append(bias)

In [ ]:
# dome flats bias subtraction
domeflats_bias = []
for file in domeflats:
    bias = subtract_bias(file, master_bias, add_keyword=True)
    domeflats_bias.append(bias)

### Master Flat

In [ ]:
def filter_fn(images):
    B = []
    V = []
    r = []
    for file in images:
        x = file.header['filter']
        if x == 'B':
            B.append(file)
        elif x == 'V':
            V.append(file)
        elif x == 'r':
            r.append(file)
    return B, V, r

def masterflat_fn(flat, flattype, filt):
    flats = Combiner(flat)
    master_flat = flats.median_combine()
    master_flat.header = flat[0].header
    master_flat.header['Combined'] = f'Combined {flattype} flats'
    master_flat.header['filter'] = f'{filt}'
    return master_flat
    
def flatcorrect_fn(images, flattype):
    i = []
    for file in images:
        x = file.header['filter']
        if x == 'B':
            flatcorrected = flat_correct(file, flattype[0])
            i.append(flatcorrected)
        elif x == 'V':
            flatcorrected = flat_correct(file, flattype[1])
            i.append(flatcorrected)
        elif x == 'r':
            flatcorrected = flat_correct(file, flattype[2])
            i.append(flatcorrected)
        else:
            continue
    return i

In [ ]:
# dome master flats
domeflats_B, domeflats_V, domeflats_r = filter_fn(domeflats_bias)
domemasterflat_B = masterflat_fn(domeflats_B, 'dome', 'B')
domemasterflat_V = masterflat_fn(domeflats_V, 'dome', 'V')
domemasterflat_r = masterflat_fn(domeflats_r, 'dome', 'r')
domemasterflats = [domemasterflat_B, domemasterflat_V, domemasterflat_r]

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(1,3,1)
plt.imshow(domemasterflat_B, vmin=np.percentile(domemasterflat_B, 5), 
           vmax=np.percentile(domemasterflat_B, 95), cmap='gray', interpolation='nearest');
plt.subplot(1,3,2)
plt.imshow(domemasterflat_V, vmin=np.percentile(domemasterflat_V, 5), 
           vmax=np.percentile(domemasterflat_V, 95), cmap='gray', interpolation='nearest');
plt.subplot(1,3,3)
plt.imshow(domemasterflat_r, vmin=np.percentile(domemasterflat_r, 5), 
           vmax=np.percentile(domemasterflat_r, 95), cmap='gray', interpolation='nearest');

In [ ]:
np.mean(domemasterflat_B), np.mean(domemasterflat_V), np.mean(domemasterflat_r),

In [ ]:
# flatcorrect_fn(images, masterflat):
# pg1633_bias
# pg1648_bias
# g9348_bias
# lilsal_bias
# ngc7318_bias

pg1633_dome = flatcorrect_fn(pg1633_bias, domemasterflats)
pg1648_dome = flatcorrect_fn(pg1648_bias, domemasterflats)
g9348_dome = flatcorrect_fn(g9348_bias, domemasterflats)
lilsal_dome = flatcorrect_fn(lilsal_bias, domemasterflats)
ngc7318_dome = flatcorrect_fn(ngc7318_bias, domemasterflats)

In [ ]:
# sky master flats
skyflats_B, skyflats_V, skyflats_r = filter_fn(skyflats)

In [ ]:
# need to combine skyflats from same days
def date_fn(images):
    _08 = []
    _09 = []
    _11 = []
    for file in images:
        date = file.header['date-obs']
        if '-08T' in date:
            _08.append(file)
        elif '-09T' in date:
            _09.append(file)
        elif '-11T' in date:
            _11.append(file)
        else:
            continue
    return _08, _09, _11

In [ ]:
# skyflats_B, skyflats_V, skyflats_r = filter_fn(skyflats)
skyflats_B_07, skyflats_B_08, _ = date_fn(skyflats_B)
skyflats_V_07, skyflats_V_08, _ = date_fn(skyflats_V)
skyflats_r_07, skyflats_r_08, _ = date_fn(skyflats_r)

skymasterflat_B_07 = masterflat_fn(skyflats_B_07, 'sky', 'B')
skymasterflat_B_08 = masterflat_fn(skyflats_B_08, 'sky', 'B')

skymasterflat_V_07 = masterflat_fn(skyflats_V_07, 'sky', 'V')
skymasterflat_V_08 = masterflat_fn(skyflats_V_08, 'sky', 'V')

skymasterflat_r_07 = masterflat_fn(skyflats_r_07, 'sky', 'r')
skymasterflat_r_08 = masterflat_fn(skyflats_r_08, 'sky', 'r')

skymasterflats_07 = [skymasterflat_B_07, skymasterflat_V_07, skymasterflat_r_07]
skymasterflats_08 = [skymasterflat_B_08, skymasterflat_V_08, skymasterflat_r_08]

In [ ]:
for x in skyflats_B_07:
    print(x.header['date-obs'])

In [ ]:
# 07
plt.figure(figsize=(10,10))
plt.subplot(1,3,1)
plt.imshow(skymasterflat_B_07, vmin=np.percentile(skymasterflat_B_07, 5), 
           vmax=np.percentile(skymasterflat_B_07, 95), cmap='inferno', interpolation='nearest');
plt.subplot(1,3,2)
plt.imshow(skymasterflat_V_07, vmin=np.percentile(skymasterflat_V_07, 5), 
           vmax=np.percentile(skymasterflat_V_07, 95), cmap='inferno', interpolation='nearest');
plt.subplot(1,3,3)
plt.imshow(skymasterflat_r_07, vmin=np.percentile(skymasterflat_r_07, 5), 
           vmax=np.percentile(skymasterflat_r_07, 95), cmap='inferno', interpolation='nearest');

In [ ]:
# 08
plt.figure(figsize=(10,10))
plt.subplot(1,3,1)
plt.imshow(skymasterflat_B_08, vmin=np.percentile(skymasterflat_B_08, 5), 
           vmax=np.percentile(skymasterflat_B_08, 95), cmap='gray', interpolation='nearest');
plt.subplot(1,3,2)
plt.imshow(skymasterflat_V_08, vmin=np.percentile(skymasterflat_V_08, 5), 
           vmax=np.percentile(skymasterflat_V_08, 95), cmap='gray', interpolation='nearest');
plt.subplot(1,3,3)
plt.imshow(skymasterflat_r_08, vmin=np.percentile(skymasterflat_r_08, 5), 
           vmax=np.percentile(skymasterflat_r_08, 95), cmap='gray', interpolation='nearest');

In [ ]:
# flatcorrect_fn(images, masterflat):
# pg1633_bias
# pg1648_bias
# g9348_bias
# lilsal_bias
# ngc7318_bias

# def date_fn(images):
#     _08 = []
#     _09 = []
#     for file in images:
#         date = file.header['date-obs']
#         if '-08T' in date:
#             _08.append(file)
#         elif '-09T' in date:
#             _09.append(file)
#     return _08, _09

# def toomanyfns(star):
#     i = []
#     for file in star:
#         date = file.header['date-obs']
#         if '-08T' in date:
#             f_07 = flatcorrect_fn(file, skymasterflats_07)
#             i.append(f_07)
#         elif '-09T' in date:
#             f_08 = flatcorrect_fn(file, skymasterflats_08)
#             i.append(f_08)
#     return i

# def flatcorrect_fn(images, flattype):
#     i = []
#     for file in images:
#         x = file.header['filter']
#         if x == 'B':
#             flatcorrected = flat_correct(file, flattype[0])
#             i.append(flatcorrected)
#         elif x == 'V':
#             flatcorrected = flat_correct(file, flattype[1])
#             i.append(flatcorrected)
#         elif x == 'r':
#             flatcorrected = flat_correct(file, flattype[2])
#             i.append(flatcorrected)
#         else:
#             continue
#     return i

# pg1633_sky = toomanyfns(pg1633_bias)

pg1633_07, pg1633_08, _ = date_fn(pg1633_bias)
pg1633_sky_07 = flatcorrect_fn(pg1633_07, skymasterflats_07)
pg1633_sky_08 = flatcorrect_fn(pg1633_08, skymasterflats_08)
pg1633_sky = pg1633_sky_07.append(pg1633_sky_08)

pg1648_07, pg1648_08, _ = date_fn(pg1648_bias)
pg1648_sky_07 = flatcorrect_fn(pg1648_07, skymasterflats_07)
pg1648_sky_08 = flatcorrect_fn(pg1648_08, skymasterflats_08)
pg1648_sky = pg1648_sky_07.append(pg1648_sky_08)

g9348_07, g9348_08, _ = date_fn(g9348_bias)
g9348_sky_07 = flatcorrect_fn(g9348_07, skymasterflats_07)
g9348_sky_08 = flatcorrect_fn(g9348_08, skymasterflats_08)
g9348_sky = g9348_sky_08

_, lilsal_08, _ = date_fn(lilsal_bias)
lilsal_sky = flatcorrect_fn(lilsal_08, skymasterflats_08)

# ngc 7318 doesn't have flats
# ngc7318_dome = flatcorrect_fn(ngc7318_bias, domemasterflats)

### Illumination Correction

In [ ]:
illum_B_07 = skymasterflats_07[0].data/domemasterflats[0].data
illum_B_08 = skymasterflats_08[0].data/domemasterflats[0].data

illum_V_07 = skymasterflats_07[1].data/domemasterflats[1].data
illum_V_08 = skymasterflats_08[1].data/domemasterflats[1].data

illum_r_07 = skymasterflats_07[2].data/domemasterflats[2].data
illum_r_08 = skymasterflats_08[2].data/domemasterflats[2].data


illum_07 = [illum_B_07, illum_V_07, illum_r_07]
illum_08 = [illum_B_08, illum_V_08, illum_r_08]

# def illumcorrect_fn(images, illumtype):
#     i = []
#     for file in images:
#         x = file.header['filter']
#         if x == 'B':
#             illumcorrected = file, illum
#             i.append(illumcorrected)
#         elif x == 'V':
#             flatcorrected = flat_correct(file, flattype[1])
#             i.append(flatcorrected)
#         elif x == 'r':
#             flatcorrected = flat_correct(file, flattype[2])
#             i.append(flatcorrected)
#         else:
#             continue
#     return i

In [ ]:
def mean_fn(star):
    i = []
    for image in star:
        meaned = image.data/np.mean(image.data)
        i.append(meaned)
    return i

# for x in pg1633_loc:
#     image = CCDData.read(x, unit='adu')
#     pg1633.append(image)

# def masterflat_fn(flat, flattype, filt):
#     flats = Combiner(flat)
#     master_flat = flats.median_combine()
#     master_flat.header = flat[0].header
#     master_flat.header['Combined'] = f'Combined {flattype} flats'
#     master_flat.header['filter'] = f'{filt}'
#     return master_flat


In [ ]:
# pg1633_dome_07, pg1633_dome_08, _ = date_fn(mean_fn(pg1633_dome))
# pg1648_dome_07, pg1648_dome_08, _ = date_fn(mean_fn(pg1648_dome))
# g9348_dome_07, g9348_dome_08, _ = date_fn(mean_fn(g9348_dome))
# lilsal_07, lilsal_08, _ = date_fn(mean_fn(lilsal_dome))

illum_mean_07 = mean_fn(illum_07)
illum_mean_08 = mean_fn(illum_08)

In [ ]:
# gaussian fn?
def illumnorm_fn(illum):
    i = []
    for x in illum:
        y = medfilt(x)
        i.append(y)
    return i

illum_norm_07 = illumnorm_fn(illum_mean_07)
illum_norm_08 = illumnorm_fn(illum_mean_08)

# plt.subplot(1, 2, 1)
# plt.imshow(illum_norm_07[0], vmin=np.percentile(illum_norm_07[0], 5), 
#            vmax=np.percentile(illum_norm_07[0], 95), cmap='gray', interpolation='nearest');
# plt.subplot(1, 2, 2)
# plt.imshow(illum_07[0], vmin=np.percentile(illum_07[0], 5), 
#            vmax=np.percentile(illum_07[0], 95), cmap='gray', interpolation='nearest');

In [ ]:
def illumcr_fn(star, illum):
    i = []
    for image in star:
        x = file.header['filter']
        if x == 'B':
            cr = image.data/illum[0]
            y = CCDData(cr, unit='adu')
            y.header = image.header
            i.append(y)
        if x == 'V':
            cr = image.data/illum[1]
            y = CCDData(cr, unit='adu')
            y.header = image.header
            i.append(y)
        if x == 'r':
            cr = image.data/illum[2]
            y = CCDData(cr, unit='adu')
            y.header = image.header
            i.append(y)
        else:
            continue
    return i

# def flatcorrect_fn(images, flattype):
#     i = []
#     for file in images:
#         x = file.header['filter']
#         if x == 'B':
#             flatcorrected = flat_correct(file, flattype[0])
#             i.append(flatcorrected)
#         elif x == 'V':
#             flatcorrected = flat_correct(file, flattype[1])
#             i.append(flatcorrected)
#         elif x == 'r':
#             flatcorrected = flat_correct(file, flattype[2])
#             i.append(flatcorrected)
#         else:
#             continue
#     return i

pg1633_07, pg1633_08, _ = date_fn(pg1633_bias)
pg1633_illum_07 = illumcr_fn(pg1633_07, illum_norm_07)
pg1633_illum_08 = illumcr_fn(pg1633_08, illum_norm_08)
# pg1633_sky = pg1633_sky_07.append(pg1633_sky_08)

pg1648_07, pg1648_08, _ = date_fn(pg1648_bias)
pg1648_illum_07 = illumcr_fn(pg1648_07, illum_norm_07)
pg1648_illum_08 = illumcr_fn(pg1648_08, illum_norm_08)
# pg1648_sky = pg1648_sky_07.append(pg1648_sky_08)

g9348_07, g9348_08, _ = date_fn(g9348_bias)
g9348_illum_07 = illumcr_fn(g9348_07, illum_norm_07)
g9348_illum_08 = illumcr_fn(g9348_08, illum_norm_08)
# g9348_sky = g9348_sky_08

_, lilsal_08, _ = date_fn(lilsal_bias)
lilsal_illum_08 = illumcr_fn(lilsal_08, illum_norm_08)

In [ ]:
plt.subplot(1, 2, 1)
plt.imshow(pg1633_07[0], vmin=np.percentile(pg1633_07[0], 5), 
           vmax=np.percentile(pg1633_07[0], 95), cmap='gray', interpolation='nearest');
plt.subplot(1, 2, 2)
plt.imshow(pg1633_illum_07[0], vmin=np.percentile(pg1633_illum_07[0], 5), 
           vmax=np.percentile(pg1633_illum_07[0], 95), cmap='gray', interpolation='nearest');

In [ ]:
print(np.median(pg1633_07[0]), np.median(pg1633_illum_07[0]))

## _Maximizing SNR_

In [ ]:
# using G9348_V_60
sn_test = g9348_sky[4]
mean, median, std = sigma_clipped_stats(sn_test, sigma=3.0)
znorm = ImageNormalize(sn_test, interval=ZScaleInterval())
print(mean, median, std)
plt.imshow(sn_test, vmin=np.percentile(sn_test, 5), 
           vmax=np.percentile(sn_test, 95), cmap='gray', interpolation='nearest');

In [ ]:
sn_test.header

In [ ]:
def makemag(flux,exptime,C):
    m = C - 2.5*np.log(np.abs(flux.value/exptime))
    return m

annulus_r_in = 15
annulus_r_out = 30

def sn_fn(aperture_radius):
    mean, median, std = sigma_clipped_stats(sn_test, sigma=3.0)
    znorm = ImageNormalize(sn_test, interval=ZScaleInterval())
    
    daofind = DAOStarFinder(fwhm=2, threshold=10*std, exclude_border=True)
    sources = daofind(sn_test.data)
    
    # aperture creation
    positions = []
    for x, y in zip(sources['xcentroid'], sources['ycentroid']):
        points = [x, y]
        positions.append(points)
    aperture = CircularAperture(positions, r=aperture_radius)
    positions = positions[7]
    aperture = aperture[7]
    
    # main star flux
    phot = aperture_photometry(sn_test, aperture)
    for col in phot.colnames:phot[col].info.format = '%.8g'
    source = phot['aperture_sum']
    
    # annulus creation
    annulus = CircularAnnulus(positions, r_in=annulus_r_in, r_out=annulus_r_out)
    total_sky = aperture_photometry(sn_test,annulus)['aperture_sum']
    avg_sky = total_sky / annulus.area
    sky = avg_sky * aperture.area
    fluxes = source - sky
    
    # magnitude calculation
    mags = makemag(fluxes,exptime,28)
    
    # signal to noise calculation
    sn = gain*fluxes.value/np.sqrt(gain*fluxes.value + gain*aperture.area*sky.value 
                            + aperture.area*read_noise**2)
    # sn = np.sqrt(gain*fluxes)
    return sn, mags

In [ ]:
radii = np.arange(1, 10, 1)
sns = []
mgs = []
for x in radii:
    sns.append(sn_fn(x)[0])
    mgs.append(sn_fn(x)[1])
    

fig, (ax0, ax1) = plt.subplots(nrows=2, sharex='col', figsize=(11, 9))

ax0.set_ylabel('Instrumental Magnitude')
ax0.plot(radii, mgs, '-o', color='b')
ax0.invert_yaxis()

ax1.set_xlabel('Aperture Radius (pixels)')
ax1.set_ylabel('SNR')
ax1.plot(radii, sns, '-o', color='k')
plt.tight_layout()

In [ ]:
daofind = DAOStarFinder(fwhm=2, threshold=10*std, exclude_border=True)
sources = daofind(sn_test.data)

positions = []
for x, y in zip(sources['xcentroid'], sources['ycentroid']):
    points = [x, y]
    positions.append(points)
positions = positions[7]
    
aperture_best = CircularAperture(positions, r=2)

aperture_smallest = CircularAperture(positions, r=1)

aperture_largest = CircularAperture(positions, r=9)

annulus = CircularAnnulus(positions, r_in=annulus_r_in, r_out=annulus_r_out)

# Plotting apertures is easy with photutils aperture objects: just add .plot
plt.figure(figsize = (11,8.5))
plt.imshow(sn_test,norm=znorm,cmap='pink')
aperture_best.plot(color='red', linewidth=2, label='"best" (r=2)')
aperture_smallest.plot(color='black', linewidth=2, label='smallest (r=1)')
aperture_largest.plot(color='lime', linewidth=2, label='largest (r=9)')
annulus.plot(color='deeppink', linewidth=2, label='annulus')
plt.legend();

In [ ]:
# Plot a 1-D model PSF
flux = 1
gamma = 1.09 # *Half* width at half max
alpha = 1 # power index (shape)

PSF = Moffat1D(flux, 0, gamma, alpha)
plt.figure(figsize=(16,6)) # make the plot w i d e
ymin, ymax = plt.ylim(-0.4,1.05) # make some space for labels
X = np.linspace(-25,25,250)
plt.plot(X, PSF(X))

# Label the fwhm
fwhm = PSF.fwhm

print("FWHM =", fwhm)
plt.hlines(0.5, 0-fwhm/2., 0+fwhm/2, color="k")
txt = plt.annotate("fwhm", (0, 0.4), size=15, ha="center")

# Show where a circular aperture would go
aperture_radius = 2
plt.hlines(-0.1, -aperture_radius, aperture_radius)
txt = plt.annotate("aperture", (0, -0.2), size=15, ha="center")

# Show sky annulus where background light is measured
plt.hlines(-0.1, -annulus_r_in, -annulus_r_out)
txt = plt.annotate("sky", (-annulus_r_out, -0.2), size=15)
plt.hlines(-0.1, annulus_r_in, annulus_r_out)
txt = plt.annotate("sky", (annulus_r_in, -0.2), size=15)

## _Extinction_

In [ ]:
lilsal_B_tot = []
lilsal_V_tot = []
for x in lilsal_sky:
    if x.header['filter'] == 'B':
        lilsal_B_tot.append(x)
    elif x.header['filter'] == 'V':
        lilsal_V_tot.append(x)
        
# for x in lilsal_V_tot:
#     print(x.header['filter'], x.header['airmass'])

In [ ]:
# choosing star to find coefficients
image = lilsal_B_tot[1]
mean, median, std = sigma_clipped_stats(image, sigma=3.0)
znorm = ImageNormalize(image, interval=ZScaleInterval())

daofind = DAOStarFinder(fwhm=2, threshold=10*std, exclude_border=True)
sources = daofind(image.data)

# aperture creation
offset = 50
positions = []
for x, y in zip(sources['xcentroid'], sources['ycentroid']):
    if (200 < x < 240 +offset) and (270 < y < 300 +offset):
        points = [x, y]
        positions.append(points)
aperture = CircularAperture(positions, r=2)
# positions = positions[6]
# aperture = aperture[6]

plt.figure(figsize = (11,8.5))
plt.imshow(image,norm=znorm,cmap='pink')
aperture.plot(color='lime', linewidth=2);

In [ ]:
phot = aperture_photometry(image, aperture)
for col in phot.colnames:phot[col].info.format = '%.8g'
source = phot['aperture_sum']
print(phot)

In [ ]:
def ext_fn(image):
    mean, median, std = sigma_clipped_stats(image, sigma=3.0)
    znorm = ImageNormalize(image, interval=ZScaleInterval())
    
    daofind = DAOStarFinder(fwhm=2, threshold=10*std, exclude_border=True)
    sources = daofind(image.data)
    
    # aperture creation
    offset = 50
    positions = []
    for x, y in zip(sources['xcentroid'], sources['ycentroid']):
        if (200 < x < 240 +offset) and (270 < y < 300 +offset):
            points = [x, y]
            positions.append(points)
    aperture = CircularAperture(positions, r=2)
    
    # main star flux
    phot = aperture_photometry(image, aperture)
    for col in phot.colnames:phot[col].info.format = '%.8g'
    source = phot['aperture_sum']
    
    # annulus creation
    annulus = CircularAnnulus(positions, r_in=annulus_r_in, r_out=annulus_r_out)
    total_sky = aperture_photometry(image,annulus)['aperture_sum']
    avg_sky = total_sky / annulus.area
    sky = avg_sky * aperture.area
    fluxes = source - sky
    
    # magnitude calculation
    mags = makemag(fluxes,exptime,0)
    return mags

In [ ]:
lilsal_B_mags = []
lilsal_B_z = []
lilsal_V_mags = []
lilsal_V_z = []
for x in lilsal_B_tot:
    lilsal_B_mags.append(ext_fn(x))
    lilsal_B_z.append(x.header['airmass'])

for x in lilsal_V_tot:
    lilsal_V_mags.append(ext_fn(x))
    lilsal_V_z.append(x.header['airmass'])

In [ ]:
B_coeffs = np.polyfit(lilsal_B_z, lilsal_B_mags, 1)
def Bfit_fn(airmass):
    Bfit_y = []
    for x in airmass:
        Bfit_y.append(B_coeffs[0]*x + B_coeffs[1])
    return Bfit_y

V_coeffs = np.polyfit(lilsal_V_z, lilsal_V_mags, 1)
def Vfit_fn(airmass):
    Vfit_y = []
    for x in airmass:
        Vfit_y.append(V_coeffs[0]*x + V_coeffs[1])
    return Vfit_y

In [ ]:
fig, ax = plt.subplots(figsize=(11,8.5))

ax.set_xlabel('Airmass')
ax.set_ylabel('Instrumental magnitude')

ax.plot(lilsal_B_z, lilsal_B_mags, 'ro', label=f'B, $k_B={B_coeffs[0].item():.2f}$')
ax.plot(lilsal_B_z, Bfit_fn(lilsal_B_z), 'r-')

ax.plot(lilsal_V_z, lilsal_V_mags, 'bo', label=f'V, $k_V={V_coeffs[0].item():.2f}$')
ax.plot(lilsal_V_z, Vfit_fn(lilsal_V_z), 'b-')

ax.invert_yaxis()
ax.legend(loc='best')
fig.tight_layout();

## _Transformation_

own train of thought here if you can see it then I forgot to delete it

-need min to max of lintest; min CANNOT HAVE THE SHUTTER IN IT  
-illum correction is weird  
-can find actual magnitudes for all targets BUUUUT don't want to, come back to it later